Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1699954449590_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1699954449590_0001,pyspark,idle,Link,Link,None,✔


### Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### Import des librairies

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [4]:
PATH = 's3://'
PATH_Data = PATH+'p8-train-data'
PATH_Result = PATH+'p8-train-data/Resultat/'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://
PATH_Data:   s3://p8-train-data
PATH_Result: s3://p8-train-data/Resultat/

## Traitement des données

### Chargement des données

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-train-dat...|2023-11-14 07:52:41|  7437|[FF D8 FF E0 00 1...|
|s3://p8-train-dat...|2023-11-14 07:52:42|  7434|[FF D8 FF E0 00 1...|
|s3://p8-train-dat...|2023-11-14 07:50:56|  7424|[FF D8 FF E0 00 1...|
|s3://p8-train-dat...|2023-11-14 07:52:41|  7423|[FF D8 FF E0 00 1...|
|s3://p8-train-dat...|2023-11-14 07:52:41|  7416|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------+--------------+
|path                                                    |label         |
+--------------------------------------------------------+--------------+
|s3://p8-train-data/Data_train/Raspberry/176_100.jpg     |Raspberry     |
|s3://p8-train-data/Data_train/Raspberry/179_100.jpg     |Raspberry     |
|s3://p8-train-data/Data_train/Pineapple Mini/170_100.jpg|Pineapple Mini|
|s3://p8-train-data/Data_train/Raspberry/157_100.jpg     |Raspberry     |
|s3://p8-train-data/Data_train/Raspberry/131_100.jpg     |Raspberry     |
+--------------------------------------------------------+--------------+
only showing top 5 rows

None

In [8]:
spark_df = images.drop("modificationTime", "length")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark_df.show(truncate=20)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------+
|                path|             content|         label|
+--------------------+--------------------+--------------+
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|Pineapple Mini|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|Pineapple Mini|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberry|
|s3://p8-train-dat...|[FF D8 FF E0 00 1...|     Raspberr

### Préparation du modèle

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
new_model.summary()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1699954449590_0001/container_1699954449590_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### Exécutions des actions d'extractions de features

In [16]:
#spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p8-train-data/Resultat/

In [19]:
df_features = spark_df.select(col("path"), col("label"), featurize_udf("content").alias("features"))
df_features.show(truncate=20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|                path|         label|            features|
+--------------------+--------------+--------------------+
|s3://p8-train-dat...|     Raspberry|[0.1676918, 0.087...|
|s3://p8-train-dat...|     Raspberry|[0.31869936, 0.14...|
|s3://p8-train-dat...|Pineapple Mini|[0.0, 5.040999, 0...|
|s3://p8-train-dat...|     Raspberry|[0.06850555, 0.00...|
|s3://p8-train-dat...|     Raspberry|[0.21430463, 0.02...|
|s3://p8-train-dat...|     Raspberry|[0.008802745, 0.2...|
|s3://p8-train-dat...|Pineapple Mini|[0.0, 4.6613655, ...|
|s3://p8-train-dat...|     Raspberry|[0.04555568, 0.16...|
|s3://p8-train-dat...|     Raspberry|[0.009401682, 0.0...|
|s3://p8-train-dat...|     Raspberry|[0.161342, 0.1989...|
|s3://p8-train-dat...|     Raspberry|[0.06575237, 0.05...|
|s3://p8-train-dat...|     Raspberry|[0.011084761, 0.2...|
|s3://p8-train-dat...|     Raspberry|[0.030955393, 0.0...|
|s3://p8-train-dat...|     Raspberry|[0.16948618, 0.06..

In [20]:
from pyspark.sql.functions import col
output_path = "s3://p8-train-data/Resultat/Results.csv"

df_string_features = df_features.withColumn("features", col("features").cast("string"))
df_string_features.write.mode("overwrite").csv(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### PCA

In [44]:
from pyspark.ml.feature import PCA

# instantiate Spark PCA model
pca2 = PCA(k=8,
          inputCol="features",
          outputCol="pca_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# fit the model on the extracted features
model2 = pca2.fit(df_features.select('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# cumulative explained variance
cumValues = model2.explainedVariance.cumsum()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# import matplotlib.pyplot as plt

# # show the scree plot
# plt.rcParams['figure.facecolor']='w'
# plt.figure(figsize=(6,3))
# plt.plot(range(1,9), cumValues, color='r',
#          marker = 'o', linestyle='--')
# plt.title('Scree plot')
# plt.xlabel('Number of first components')
# plt.ylabel('Cumulative explained variance')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# get the projection of the extracted features using PCA

pca_feat_df = model2.transform(df_features.select('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# Affichez les résultats
pca_feat_df.select("features", "pca_features").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
# Enregistrement du DataFrame transformé au format Parquet sur Amazon S3
output_path_pca_parquet = "s3://p8-train-data/Resultat/Results_PCA.parquet"

# Utilisation de la méthode write pour enregistrer le DataFrame au format Parquet
pca_feat_df.write.parquet(output_path_pca_parquet, mode='overwrite')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### PCA

In [21]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StringIndexer, StandardScaler
from pyspark.ml.feature import PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Définition de la fonction UDF
convert_to_vector = udf(lambda l: Vectors.dense(l), VectorUDT())

# Application de la fonction UDF et ajout de la colonne 'features'
df_features = df_features.withColumn('features', convert_to_vector(df_features["features"]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Création de l'objet StandardScaler
standardizer = StandardScaler(inputCol='features', outputCol='feats_scaled', withStd=True, withMean=False)
std_model = standardizer.fit(df_features)
df_features_scaled = std_model.transform(df_features).drop("features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_features_scaled

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, feats_scaled: vector]

In [25]:
# Application de PCA
pca = PCA(k=2, inputCol='feats_scaled', outputCol='pca')
model_pca = pca.fit(df_features_scaled)
transformed = model_pca.transform(df_features_scaled)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…